In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# use pip install tensorflow-datasets
import tensorflow_datasets as tfds

In [13]:
# conda install -c conda-forge ipywidgets {to install ipywidgets}
# jupyter nbextension enable --py widgetsnbextension {to enable nbextension}
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [14]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [17]:
mnist_train.output_types

(tf.uint8, tf.int64)

In [5]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

In [6]:
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    
    return image, label

In [8]:
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [9]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [10]:
BATCH_SIZE = 150

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

In [12]:
train_data.head()

AttributeError: 'BatchDataset' object has no attribute 'head'

In [11]:
validation_inputs, validation_targets = next(iter(validation_data))

In [20]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    
    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into our net, so we must flatten the images
    # there is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) 
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 3rd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
NUM_EPOCHS = 5

# we fit the model, specifying the
# training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
360/360 - 9s - loss: 0.3707 - accuracy: 0.8916 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
360/360 - 9s - loss: 0.1394 - accuracy: 0.9588 - val_loss: 0.1273 - val_accuracy: 0.9630
Epoch 3/5
360/360 - 9s - loss: 0.0995 - accuracy: 0.9699 - val_loss: 0.0994 - val_accuracy: 0.9695
Epoch 4/5
360/360 - 10s - loss: 0.0778 - accuracy: 0.9761 - val_loss: 0.0781 - val_accuracy: 0.9773
Epoch 5/5
360/360 - 11s - loss: 0.0610 - accuracy: 0.9812 - val_loss: 0.0650 - val_accuracy: 0.9795


In [23]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.0829 - accuracy: 0.97 - 2s 2s/step - loss: 0.0829 - accuracy: 0.9739

In [24]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.39%
